In [449]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
from sqlalchemy.types import VARCHAR

rds_connection_string = "root:AFg167769@127.0.0.1/Opioid_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [450]:
#loading csv file
county_data = "2016_Opioid_County.csv"

In [451]:
#reading in data
county_opioid = pd.read_csv(county_data)

In [452]:
#sneakpeak 
#county_opioid

In [453]:
#cleaning data up to have correct row identified as columns, renaming df as county_df 
headers = county_opioid.iloc[3]
county_df  = pd.DataFrame(county_opioid.values[4:], columns=headers)
county_df = county_df.drop(county_df.index[0])
county_df = county_df.dropna()
county_df = county_df.reset_index()
county_df.drop(['index'], axis=1, inplace=True)
county_df

3,State Name,State Abbreviation,County Name,FIPS,Part D Prescribers,Part D Opioid Prescribers,Opioid Claims,Extended Release Opioid Claims,Overall Claims,Opioid Prescribing Rate,Extended Release Opioid Prescribing Rate,Change in Opioid Prescribing Rate,Change in Extended Release Opioid Prescribing Rate
0,Alabama,AL,Autauga,01001,73,45,"24,358","2,286","212,377",11.47,9.39,1.67,4.61
1,Alabama,AL,Baldwin,01003,457,259,"80,557","6,489","940,727",8.56,8.06,-0.58,0.33
2,Alabama,AL,Barbour,01005,31,21,"4,383",165,"123,196",3.56,3.76,-2.72,1.49
3,Alabama,AL,Bibb,01007,40,14,"1,759",43,"47,162",3.73,2.44,-0.25,0.52
4,Alabama,AL,Blount,01009,26,21,"7,846",56,"135,749",5.78,0.71,-0.34,-1.00
5,Alabama,AL,Bullock,01011,7,6,"1,920",40,"40,333",4.76,2.08,1.02,0.96
6,Alabama,AL,Butler,01013,36,25,"3,943",26,"93,106",4.23,0.66,-3.96,-0.39
7,Alabama,AL,Calhoun,01015,276,159,"47,278","1,759","645,365",7.33,3.72,-0.65,-1.77
8,Alabama,AL,Chambers,01017,32,22,"10,700",647,"136,055",7.86,6.05,-0.10,-0.14
9,Alabama,AL,Cherokee,01019,32,20,"13,189",934,"168,763",7.82,7.08,-0.34,5.08


In [454]:
unemployment_data = "Unemployment_by_county.csv"
unemployment_by_county = pd.read_csv(unemployment_data)
#unemployment_by_county 

In [455]:
#selecting correct row to be column names 
headers = unemployment_by_county.iloc[3]
labor_county_df  = pd.DataFrame(unemployment_by_county.values[5:], columns=headers)


In [456]:
labor_county_df.columns = labor_county_df.columns.fillna('nan')
labor_county_df.drop('nan', axis = 1, inplace = True)

In [457]:
labor_county_df.drop('Code', axis= 1, inplace = True) 

In [458]:
#labor_county_df.head() 

In [459]:
#labor_county_df['County Name/State Abbreviation'].str.split(' ', 2, expand=True)

In [460]:
#getting rid of county
labor_county_df['County Name/State Abbreviation'] = labor_county_df['County Name/State Abbreviation'].str.replace('County', '')

In [461]:
#getting rid of municipio 
labor_county_df['County Name/State Abbreviation'] = labor_county_df['County Name/State Abbreviation'].str.replace('Municipio', '')

In [462]:
#labor_county_df

In [463]:
#getting rid of , 
state_county = labor_county_df['County Name/State Abbreviation'].str.split(',', 2, expand=True)

In [464]:
state_county 
merged_labor_county = pd.merge(state_county, labor_county_df, left_index=True, right_index=True)
merged_labor_county = merged_labor_county.drop(['County Name/State Abbreviation'], axis=1)

In [465]:
merged_labor_county.columns = ['County Name', 'State Abbreviation', 'Year', 'Workforce', 'Employed', 'Unemployed', '(%)'] 

In [466]:
#merged_labor_county

In [467]:
#making sure whitespace isn't a problem 
merged_labor_county = merged_labor_county.rename(columns={'County Name': 'County Name'})
county_df = county_df.rename(columns={'County Name': 'County Name'})

In [468]:
#making sure theres no whitespace causing problems 
merged_labor_county['County Name'] = merged_labor_county['County Name'].str.strip()
merged_labor_county['State Abbreviation'] = merged_labor_county['State Abbreviation'].str.strip()
county_df['County Name'] = county_df['County Name'].str.strip()
county_df['State Abbreviation'] = county_df['State Abbreviation'].str.strip() 

In [469]:
merged_labor_county = merged_labor_county.dropna()
#merged_labor_county

In [470]:
#merging opiate data with unemployment 
merge = pd.merge(merged_labor_county, county_df, on=["County Name", "State Abbreviation"])

In [471]:
merge

,County Name,State Abbreviation,Year,Workforce,Employed,Unemployed,(%),State Name,FIPS,Part D Prescribers,Part D Opioid Prescribers,Opioid Claims,Extended Release Opioid Claims,Overall Claims,Opioid Prescribing Rate,Extended Release Opioid Prescribing Rate,Change in Opioid Prescribing Rate,Change in Extended Release Opioid Prescribing Rate
0,Autauga,AL,2016,"25,918","24,593","1,325",5.1,Alabama,01001,73,45,"24,358","2,286","212,377",11.47,9.39,1.67,4.61
1,Baldwin,AL,2016,"90,500","85,656","4,844",5.4,Alabama,01003,457,259,"80,557","6,489","940,727",8.56,8.06,-0.58,0.33
2,Barbour,AL,2016,"8,402","7,700",702,8.4,Alabama,01005,31,21,"4,383",165,"123,196",3.56,3.76,-2.72,1.49
3,Bibb,AL,2016,"8,607","8,050",557,6.5,Alabama,01007,40,14,"1,759",43,"47,162",3.73,2.44,-0.25,0.52
4,Blount,AL,2016,"24,576","23,248","1,328",5.4,Alabama,01009,26,21,"7,846",56,"135,749",5.78,0.71,-0.34,-1.00
5,Bullock,AL,2016,"4,824","4,493",331,6.9,Alabama,01011,7,6,"1,920",40,"40,333",4.76,2.08,1.02,0.96
6,Butler,AL,2016,"9,190","8,553",637,6.9,Alabama,01013,36,25,"3,943",26,"93,106",4.23,0.66,-3.96,-0.39
7,Calhoun,AL,2016,"45,818","42,815","3,003",6.6,Alabama,01015,276,159,"47,278","1,759","645,365",7.33,3.72,-0.65,-1.77
8,Chambers,AL,2016,"14,978","14,158",820,5.5,Alabama,01017,32,22,"10,700",647,"136,055",7.86,6.05,-0.10,-0.14
9,Cherokee,AL,2016,"11,332","10,757",575,5.1,Alabama,01019,32,20,"13,189",934,"168,763",7.82,7.08,-0.34,5.08


In [472]:
# dropping unwanted columns 
merge_clean = merge.drop(['Workforce','Employed','Unemployed','Part D Prescribers','Part D Opioid Prescribers'], axis=1)


In [473]:
#showing merge that is cleaned up 
merge_clean
merge_clean = merge_clean.reset_index(drop=True)
merge_clean

,County Name,State Abbreviation,Year,(%),State Name,FIPS,Opioid Claims,Extended Release Opioid Claims,Overall Claims,Opioid Prescribing Rate,Extended Release Opioid Prescribing Rate,Change in Opioid Prescribing Rate,Change in Extended Release Opioid Prescribing Rate
0,Autauga,AL,2016,5.1,Alabama,01001,"24,358","2,286","212,377",11.47,9.39,1.67,4.61
1,Baldwin,AL,2016,5.4,Alabama,01003,"80,557","6,489","940,727",8.56,8.06,-0.58,0.33
2,Barbour,AL,2016,8.4,Alabama,01005,"4,383",165,"123,196",3.56,3.76,-2.72,1.49
3,Bibb,AL,2016,6.5,Alabama,01007,"1,759",43,"47,162",3.73,2.44,-0.25,0.52
4,Blount,AL,2016,5.4,Alabama,01009,"7,846",56,"135,749",5.78,0.71,-0.34,-1.00
5,Bullock,AL,2016,6.9,Alabama,01011,"1,920",40,"40,333",4.76,2.08,1.02,0.96
6,Butler,AL,2016,6.9,Alabama,01013,"3,943",26,"93,106",4.23,0.66,-3.96,-0.39
7,Calhoun,AL,2016,6.6,Alabama,01015,"47,278","1,759","645,365",7.33,3.72,-0.65,-1.77
8,Chambers,AL,2016,5.5,Alabama,01017,"10,700",647,"136,055",7.86,6.05,-0.10,-0.14
9,Cherokee,AL,2016,5.1,Alabama,01019,"13,189",934,"168,763",7.82,7.08,-0.34,5.08


In [474]:
#listed columns to find problem--there was a space after last column before quotation
list(merge_clean.columns.values)

['County Name',
 'State Abbreviation',
 'Year',
 '(%)',
 'State Name',
 'FIPS',
 'Opioid Claims',
 'Extended Release Opioid Claims',
 'Overall Claims',
 'Opioid Prescribing Rate',
 'Extended Release Opioid Prescribing Rate',
 'Change in Opioid Prescribing Rate',
 'Change in Extended Release Opioid Prescribing Rate ']

In [475]:
#reording and selecting columns 
merge_clean = merge_clean[['Year','County Name','State Name','Opioid Claims',
                         'Extended Release Opioid Claims','Overall Claims','Opioid Prescribing Rate',
                         'Extended Release Opioid Prescribing Rate','Change in Opioid Prescribing Rate',
                         'Change in Extended Release Opioid Prescribing Rate ','(%)']] 
merge_clean.rename(columns={'(%)': 'Rate of Unemployment'}, inplace=True)
merge_clean.rename(columns={'Change in Extended Release Opioid Prescribing Rate ': 'Change in Extended Release Opioid Prescribing Rate'}, inplace=True) 

/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [476]:
merge_clean

,Year,County Name,State Name,Opioid Claims,Extended Release Opioid Claims,Overall Claims,Opioid Prescribing Rate,Extended Release Opioid Prescribing Rate,Change in Opioid Prescribing Rate,Change in Extended Release Opioid Prescribing Rate,Rate of Unemployment
0,2016,Autauga,Alabama,"24,358","2,286","212,377",11.47,9.39,1.67,4.61,5.1
1,2016,Baldwin,Alabama,"80,557","6,489","940,727",8.56,8.06,-0.58,0.33,5.4
2,2016,Barbour,Alabama,"4,383",165,"123,196",3.56,3.76,-2.72,1.49,8.4
3,2016,Bibb,Alabama,"1,759",43,"47,162",3.73,2.44,-0.25,0.52,6.5
4,2016,Blount,Alabama,"7,846",56,"135,749",5.78,0.71,-0.34,-1.00,5.4
5,2016,Bullock,Alabama,"1,920",40,"40,333",4.76,2.08,1.02,0.96,6.9
6,2016,Butler,Alabama,"3,943",26,"93,106",4.23,0.66,-3.96,-0.39,6.9
7,2016,Calhoun,Alabama,"47,278","1,759","645,365",7.33,3.72,-0.65,-1.77,6.6
8,2016,Chambers,Alabama,"10,700",647,"136,055",7.86,6.05,-0.10,-0.14,5.5
9,2016,Cherokee,Alabama,"13,189",934,"168,763",7.82,7.08,-0.34,5.08,5.1


In [477]:
#group by state, calculate columns 
#maybe create bins for unemployment rates and groupby unemployment rates

In [478]:

merge_clean["Opioid Claims"] = merge_clean["Opioid Claims"].str.replace(",","").astype(float)
merge_clean["Extended Release Opioid Claims"] = merge_clean["Extended Release Opioid Claims"].str.replace(",","").astype(float)
merge_clean["Overall Claims"] = merge_clean["Overall Claims"].str.replace(",","").astype(float)

/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [479]:
merge_clean["Rate of Unemployment"] = pd.to_numeric(merge_clean["Rate of Unemployment"])
merge_clean["Change in Extended Release Opioid Prescribing Rate"] = pd.to_numeric(merge_clean["Change in Extended Release Opioid Prescribing Rate"])
merge_clean["Change in Opioid Prescribing Rate"] = pd.to_numeric(merge_clean["Change in Opioid Prescribing Rate"])
merge_clean["Extended Release Opioid Prescribing Rate"] = pd.to_numeric(merge_clean["Extended Release Opioid Prescribing Rate"])
merge_clean["Opioid Prescribing Rate"] = pd.to_numeric(merge_clean["Opioid Prescribing Rate"])
merge_clean["Overall Claims"] = pd.to_numeric(merge_clean["Overall Claims"])
merge_clean["Extended Release Opioid Claims"] = pd.to_numeric(merge_clean["Extended Release Opioid Claims"])
merge_clean["Opioid Claims"] = pd.to_numeric(merge_clean["Opioid Claims"]) 

/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [480]:

merge_clean.dtypes

Year                                                   object
County Name                                            object
State Name                                             object
Opioid Claims                                         float64
Extended Release Opioid Claims                        float64
Overall Claims                                        float64
Opioid Prescribing Rate                               float64
Extended Release Opioid Prescribing Rate              float64
Change in Opioid Prescribing Rate                     float64
Change in Extended Release Opioid Prescribing Rate    float64
Rate of Unemployment                                  float64
dtype: object

In [481]:
state_group = merge_clean.groupby('State Name')

state_group.count().head(10) 

,Year,County Name,Opioid Claims,Extended Release Opioid Claims,Overall Claims,Opioid Prescribing Rate,Extended Release Opioid Prescribing Rate,Change in Opioid Prescribing Rate,Change in Extended Release Opioid Prescribing Rate,Rate of Unemployment
State Name,,,,,,,,,,
Alabama,67,67,67,67,67,67,67,67,67,67
Arizona,15,15,15,15,15,15,15,15,15,15
Arkansas,75,75,75,75,75,75,75,75,75,75
California,56,56,56,56,56,56,56,56,56,56
Colorado,60,60,60,60,60,60,60,60,60,60
Connecticut,8,8,8,8,8,8,8,8,8,8
Delaware,3,3,3,3,3,3,3,3,3,3
Florida,67,67,67,67,67,67,67,67,67,67
Georgia,155,155,155,155,155,155,155,155,155,155


In [482]:
opioid_claims = state_group['Opioid Claims'].sum() 
extended_claims = state_group['Extended Release Opioid Claims'].sum()
overall_claims = state_group['Overall Claims'].sum()
opioid_rate = state_group['Opioid Prescribing Rate'].mean()
extended_rate = state_group['Extended Release Opioid Prescribing Rate'].mean() 
change_rate = state_group['Change in Opioid Prescribing Rate'].mean() 
extended_rate_change = state_group['Change in Extended Release Opioid Prescribing Rate'].mean()
rate_of_unemployment = state_group['Rate of Unemployment'].mean()

In [483]:
state_group_df = pd.DataFrame({'Opioid Claims': opioid_claims,
                                    'Extended Release Opioid Claims': extended_claims,
                                    'Overall Claims': overall_claims,
                                    'Opioid Prescribing Rate': opioid_rate,
                              'Change in Opioid Prescribing Rate': change_rate, 
                              'Extended Release Opioid Prescribing Rate': extended_rate,
                              'Change in Opioid Prescribing Rate': change_rate,
                              'Change in Extended Release Opioid Prescribing Rate': extended_rate_change,
                              'Rate of Unemployment': rate_of_unemployment}) 

In [484]:
state_group_df

,Opioid Claims,Extended Release Opioid Claims,Overall Claims,Opioid Prescribing Rate,Change in Opioid Prescribing Rate,Extended Release Opioid Prescribing Rate,Change in Extended Release Opioid Prescribing Rate,Rate of Unemployment
State Name,,,,,,,,
Alabama,2120450.0,119199.0,28430590.0,6.823433,-0.406567,3.690597,0.329851,6.700000
Arizona,1603895.0,171274.0,24653623.0,6.204667,-0.829333,9.508667,0.616000,7.653333
Arkansas,1152451.0,65752.0,18111202.0,5.611867,-0.578800,3.086267,-0.225467,4.606667
California,6802163.0,513487.0,134513909.0,6.870536,-0.759107,8.744286,0.620536,7.001786
Colorado,1020116.0,92294.0,14527297.0,7.404500,-0.242833,7.921833,1.752000,3.365000
Connecticut,643766.0,42984.0,15848198.0,4.217500,-0.527500,6.621250,1.227500,5.025000
Delaware,228003.0,24772.0,3877879.0,5.853333,-0.680000,11.200000,2.340000,4.600000
Florida,5534487.0,475469.0,106404520.0,5.205672,-0.728806,5.901343,0.568060,5.080597
Georgia,2799815.0,169212.0,47126808.0,5.259355,-0.773548,3.452903,0.260323,6.050323


In [485]:
merge_clean.to_sql('County_Opioid__vs_Unemployment_Data', engine, if_exists='replace')
state_group_df.to_sql('State_Opioid_vs_Unemployment_Data', engine, if_exists='replace', index_label='index', dtype={state_group_df.index.name:VARCHAR(20)})


